### Practica 12 Clasificación categórica con Naive Bayes

Utilizaremos el dataset "Mushroom" que contiene  datos de setas comestibles y venenosas .\
Contiene características de los géneros Agaricus (comestibles) y Lepiota (tóxicos).

Los atributos predictores son todos categóricos y sus descripciones son las siguientes para el dataset Mushroom:

forma_sombrero: Forma del sombrero de la seta (campana, cónica, convexa, plana, nudosa, etc.).\
superficie_sombrero: Textura de la superficie del sombrero (fibrosa, escamosa, lisa, etc.).\
color_sombrero: Color del sombrero (marrón, amarillo, blanco, rojo, etc.).\
magulladuras: Indica si la seta tiene magulladuras o no.\
olor: Olor característico de la seta (almendras, anís, pescado, moho, etc.).\
adjunto_laminas: Modo en que las láminas están adheridas al tallo.\
espaciado_laminas: Espaciado entre las láminas (estrecho o amplio).\
tamano_laminas: Tamaño de las láminas (grande o pequeño).\
color_laminas: Color de las láminas (negro, marrón, amarillo, blanco, etc.).\
forma_tallo: Forma del tallo (ensanchado, alargado, etc.).\
raiz_tallo: Tipo de raíz del tallo (bulbosa, fibrosa, sin raíz, etc.).\
superficie_tallo_arriba_anillo: Textura del tallo por encima del anillo.\
superficie_tallo_abajo_anillo: Textura del tallo por debajo del anillo.\
color_tallo_arriba_anillo: Color del tallo por encima del anillo.\
color_tallo_abajo_anillo: Color del tallo por debajo del anillo.\
tipo_vestidura: Tipo de velo que cubre la seta.\
color_vestidura: Color de la vestidura.\
numero_anillos: Número de anillos en el tallo.\
tipo_anillo: Tipo de anillo presente en el tallo.\
color_esporas: Color de la impresión de esporas de la seta.\
poblacion: Tipo de crecimiento de la seta (solitaria, agrupada, numerosa, etc.).\
habitat: Lugar donde se encuentra la seta (bosques, prados, caminos, etc.).

La clase indica si una seta es comestible o venenosa atendiendo a sus caracteristicas predictoras.

El LabelEncoder asigna valores numéricos a las categorías en orden lexicográfico (alfabético ascendente). Es decir, las etiquetas se ordenan primero alfabéticamente y luego se asignan los valores empezando desde 0.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, classification_report

# Cargar el dataset Mushroom
dataset_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"
column_names = ["clase", "forma_sombrero", "superficie_sombrero", "color_sombrero", "magulladuras", "olor", "adjunto_laminas", 
                "espaciado_laminas", "tamano_laminas", "color_laminas", "forma_tallo", "raiz_tallo", "superficie_tallo_arriba_anillo", 
                "superficie_tallo_abajo_anillo", "color_tallo_arriba_anillo", "color_tallo_abajo_anillo", "tipo_vestidura", "color_vestidura", 
                "numero_anillos", "tipo_anillo", "color_esporas", "poblacion", "habitat"]

df = pd.read_csv(dataset_url, names=column_names)

print("Filas y columnas del dataset: ",df.shape)
print()

# Transforma variables categóricas a números
label_encoders = {}
for column in df.columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le  # Conservo la informacion de asignacion etiqueta-valor

# Visualizar la asignación. Toma solo la inicial del valor de la etiqueta
print("Transformación de etiquetas efectuada")
print()
for column, le in label_encoders.items():
    print(f"Columna: {column}")
    print(f"Asignaciones: {dict(zip(le.classes_, le.transform(le.classes_)))}\n")

# Separar variables predictoras y objetivo
X = df.drop(columns=["clase"])
y = df["clase"]

# División en entrenamiento (75%) y validación (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Crear y entrenar el clasificador Naive Bayes categórico
clf = CategoricalNB()
clf.fit(X_train, y_train)

# Predicción en el conjunto de validación
y_pred = clf.predict(X_test)

# Matriz de confusión y matriz de confusion normalizada
cm = confusion_matrix(y_test, y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Función para mostrar matrices de confusión
def plot_confusion_matrix(cm, title, cmap=plt.cm.Blues):
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='.2f', cmap=plt.cm.Blues, xticklabels=['Venenosa', 'Comestible'], yticklabels=['Venenosa', 'Comestible'])
    plt.xlabel('Predicción')
    plt.ylabel('Real')
    plt.title(title)
    plt.show()

# Mostrar matrices de confusión
plot_confusion_matrix(cm, "Matriz de Confusión Absoluta")
plot_confusion_matrix(cm_norm, "Matriz de Confusión Normalizada")

# Mostrar Informe de clasificación
print("Informe de Clasificación:\n", classification_report(y_test, y_pred, target_names=['Venenosa', 'Comestible']))

#----------------------------------------------------------------
# Función para clasificar un nuevo ejemplo no visto
# Debo generar los valores numericos partiendo de los utilizados en la codificacion inicial con el mismo LabelEncoder
def clasificar_nuevo_ejemplo(nuevo_ejemplo):
    
    nuevo_ejemplo_encoded = []
    for i, column in enumerate(X.columns):
        nuevo_ejemplo_encoded.append(label_encoders[column].transform([nuevo_ejemplo[i]])[0])
    
    # Aquí tenemos las etiquetas convertidas a valores junto con el nombre de las columnas
    nuevo_ejemplo_encoded = pd.DataFrame([nuevo_ejemplo_encoded], columns=X.columns) 

    # Predecir la clase y obtener las probabilidades
    prediccion = clf.predict(nuevo_ejemplo_encoded)
    probabilidades = clf.predict_proba(nuevo_ejemplo_encoded)
    
    # Mostrar resultados
    clase_predicha = "Venenosa" if prediccion[0] == 0 else "Comestible"
    print(f"Clase predicha: {clase_predicha}")
    print(f"Probabilidades posteriores: {probabilidades}")

# Ejemplo de uso con valores ficticios (deben ser valores categóricos del dataset original)
nuevo_ejemplo = ["x", "s", "n", "t", "p", "f", "c", "n", "k", "e", "e", "s", "s", "w", "w", "p", "w", "o", "p", "n", "n", "g"]
clasificar_nuevo_ejemplo(nuevo_ejemplo)

 Ejercicio: Añade al siguiente programa Python (tomando como referencia el programa anterior) el codigo necesario para visualizar las matrices de confusión y el informe de clasificación

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Cargar el dataset Titanic
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Preprocesamiento: selección de características relevantes
df = df[['Survived', 'Pclass', 'Sex', 'Embarked', 'Age', 'SibSp', 'Parch']]

# Manejo de valores nulos: reemplazar por la moda en columnas categóricas y media en Age
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Convertir columnas categóricas a valores numéricos
label_encoders = {}
for column in ['Sex', 'Embarked']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Convertir "Age" a categórica y luego a numérica
age_labels = ['child', 'teen', 'adult', 'middle_age', 'senior']
df['Age'] = pd.cut(df['Age'], bins=[0, 12, 20, 40, 60, 100], labels=age_labels)

# Convertir las categorías de "Age" en valores numéricos
age_mapping = {'child': 0, 'teen': 1, 'adult': 2, 'middle_age': 3, 'senior': 4}
df['Age'] = df['Age'].map(age_mapping)

# Dividir los datos en entrenamiento y validación
X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)




Ejercicio: Partiendo del dataset weather.nominal.csv que contiene información sobre condiciones meteorológicas, completar el código siguiente para entrenar un clasificador categórico y predecir si un día determinado un grupo de amigos quedan para una partida de baloncesto al aire libre.

Hay que entrenar con todos los datos y hay que obtener las métricas con todos los datos. Se pide hacerlo así, y no partir el dataset en uno de entrenamiento y otro de validación porque este es muy pequeño.

Muestra las matrices de confusión.

Muestra el classification_report, y coteja los datos que muestra con los mostrados en las matrices de confusión anteriores.

Añade un nuevo dato no visto antes para ver la predicción.

Toma la práctica 12 como referencia

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar el dataset desde el archivo CSV
url = 'https://raw.githubusercontent.com/dataprofessor/data/master/weather-nominal-weka.csv'
df = pd.read_csv(url)

# Cambiar los nombres de las columnas a español
df.columns = ['clima', 'temperatura', 'humedad', 'viento', 'jugar']
